In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.metrics import confusion_matrix, mean_squared_error , r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Read Data

In [ ]:
data_2c = pd.read_csv("/kaggle/input/biomechanical-features-of-orthopedic-patients/column_2C_weka.csv")

In [ ]:
data_2c.info()

In [ ]:
data_2c.rename(columns={"pelvic_tilt numeric" : "pelvic_tilt"} , inplace=True)
data_2c.columns

In [ ]:
data_2c.describe().T

In [ ]:
data_2c.groupby(by="class").agg(["min","mean","max"]).T

In [ ]:
data_2c.isnull().sum()

## Visualization

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot("sacral_slope" , y="pelvic_incidence" , hue="class" , data=data_2c);

In [ ]:
sns.pairplot(data_2c, hue="class");

## Preprocessing

 ### Label Encoding

In [ ]:
le = LabelEncoder()
data_2c["class"] = le.fit_transform(data_2c["class"])

### Data Split & Scaling

In [ ]:
X = data_2c.drop(["class"] , axis=1)
y = data_2c["class"].values

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

## Create Model

In [ ]:
model = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

## Model Tuning

In [ ]:
test_score_list = []
train_score_list=[]
neigh = np.arange(1,20)
for each in neigh:
    model = KNeighborsClassifier(n_neighbors=each)
    model = model.fit(X_train, y_train)
    test_score_list.append(model.score(X_test,y_test))
    train_score_list.append(model.score(X_train,y_train))
plt.figure(figsize=(10,5))
plt.plot(neigh, test_score_list , color="green" , label="Test Score")
plt.plot(neigh, train_score_list , color="red" , label="Train Score")
plt.xlabel("K value")
plt.ylabel("Model Score")
plt.xticks(neigh)
plt.legend()
plt.show()

In [ ]:
print("Best Score : {}".format(np.max(test_score_list)))

In [ ]:
 model_tuned = KNeighborsClassifier(n_neighbors=19).fit(X_train, y_train)

## Prediction

In [ ]:
y_pred = model_tuned.predict(X_test)

In [ ]:
print("MSE : {}".format(mean_squared_error(y_test, y_pred)))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm , annot=True, cbar=False);